In [1]:
import numpy as np
import math
import tensorflow as tf


In [11]:
x=str(-1)
n=x[1:]
y=int(n)
print(y)


1


In [13]:
x=tf.constant([[0.0034,-0.12, 0.045, 0.2, 1, -1.05, 2.34, -0.44, 0.5],[0.0034,-0.12, 0.045, 0.2, 1, -1.05, 2.34, -0.44, 0.5]])
y=tf.reduce_max(x)
c=x/y
c

<tf.Tensor: shape=(2, 9), dtype=float32, numpy=
array([[ 0.00145299, -0.05128205,  0.01923077,  0.08547009,  0.42735043,
        -0.44871795,  1.        , -0.18803419,  0.21367522],
       [ 0.00145299, -0.05128205,  0.01923077,  0.08547009,  0.42735043,
        -0.44871795,  1.        , -0.18803419,  0.21367522]],
      dtype=float32)>

In [2]:
1/225


0.0044444444444444444

In [1]:
def LQ(tensor, bitwidth):
    # Create a mask for weights equal to 0
    #rounded_tensor=tf.where(tensor>0,tf.my u4xdcvp]
    # [6ath.floor(tensor),tf.math.ceil(tensor))
    rounded_weights=tensor*100
    rounded_weights=tf.where(rounded_weights>0,tf.math.floor(rounded_weights),tf.math.ceil(rounded_weights))
    epsilon = 1e-7

    # Compute weight_log using log2
    weight_log = tf.math.log(tf.math.abs(tensor)+epsilon) / tf.math.log(2.0)
    print(weight_log)
    weight_log=tf.where(weight_log<0,tf.math.ceil(weight_log),tf.math.floor(weight_log))
    bit_pot = -2 ** bitwidth
    mask_condition = weight_log >= bit_pot
    
    # Initialize the result tensor with zeros
    result = tf.zeros_like(tensor, dtype=tf.float32)
    
    # Apply the condition where weight_log is valid
    result = tf.where(rounded_weights!=0,tf.where(mask_condition, tf.where(tensor > 0, tf.math.abs(weight_log), weight_log), result),result)
    
    return result



In [14]:
lq=LQ(c,4)
lq


tf.Tensor(
[[-9.4266586e+00 -4.2853994e+00 -5.7004318e+00 -3.5484347e+00
  -1.2265083e+00 -1.1561189e+00  1.7198263e-07 -2.4109323e+00
  -2.2265079e+00]
 [-9.4266586e+00 -4.2853994e+00 -5.7004318e+00 -3.5484347e+00
  -1.2265083e+00 -1.1561189e+00  1.7198263e-07 -2.4109323e+00
  -2.2265079e+00]], shape=(2, 9), dtype=float32)


<tf.Tensor: shape=(2, 9), dtype=float32, numpy=
array([[ 0., -4.,  5.,  3.,  1., -1.,  0., -2.,  2.],
       [ 0., -4.,  5.,  3.,  1., -1.,  0., -2.,  2.]], dtype=float32)>

In [9]:
tf.shape(c)[0]

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [4]:
def bitshift(weight,shift):
   res=0
   if shift<0:
       y=np.abs(shift)
       res= weight>>shift
   else:
        res= weight<<shift
   def grad(dy):
       dw=2**shift*dy
       ds=2**shift*weight*tf.math.log(2)*dy
       return dw,ds
   return res,grad

In [2]:
def bitshift2(weight,shift):
    if shift<0:
       y=tf.math.abs(shift)
       return tf.bitwise.right_shift(weight,shift)
    else:
       return tf.bitwise.left_shift(weight,shift)


In [3]:
def MatrixBitShift(A,B):
    A=tf.cast(A,tf.int32)
    B=tf.cast(B,tf.int32)
    columns2=tf.shape(B)[1]
    rows2=tf.shape(B)[0]
    rows1=tf.shape(A)[0]
    C=tf.zeros(shape=(rows1,columns2),dtype=tf.int32)
    for i in range(0,rows1):
      for j in range(0,rows2):
            for  k in range(0,columns2):
                original=C[i][k]
                a=A[i][j]
                b=B[j][k]
                x=bitshift2(a,b)
                #print(x)
                update=original+x
                
                index=tf.constant([[i,k]])
                #u=tf.constant([update],dtype=tf.int32)
                C=tf.tensor_scatter_nd_update(C,index,[update])
    
    return C
    

In [6]:
tensor = tf.constant([[1, 1, 2, 3],[1,1,2,3]])
tens2=tf.constant([[1,1,1],[1,1,1],[1,1,1],[1,1,1]])
C=MatrixBitShift(tensor,tens2)
print(C)
#print(tensor[0][1])

tf.Tensor(
[[14 14 14]
 [14 14 14]], shape=(2, 3), dtype=int32)


In [4]:
import os 
os.chdir("Air Dataset")

In [6]:
x=np.ones(1024, dtype=int)
with open('dense_biases.txt', 'w') as cf:
       np.savetxt(cf,x,delimiter=" ",fmt="%d")


In [7]:
def float_to_fixed_16bit_q15(value):
    # Define the scaling factor for Q0.15 format (15 fractional bits)
    scaling_factor = 2 ** 31
    
    # Scale the floating-point value
    scaled_value = int(round(value * scaling_factor))
    
    # Define the range for a 16-bit signed integer in Q0.15 format
    min_val = -(2 ** 31)
    max_val = (2 ** 31) - 1
    
     #Clamp the value to fit within the 16-bit signed integer range
    
    if scaled_value < min_val:
        scaled_value = min_val
    elif scaled_value > max_val:
        scaled_value = max_val
    
    return scaled_value


In [8]:
float_value = 1/255
fixed_value_q15 = float_to_fixed_16bit_q15(float_value)
fixed_value_q15


8421505